# Deep One Class Classification using Contradictions (DOC3)

## Imports

In [1]:
import os
import time
import torch
import logging
import torchvision
import numpy as np
import torch.nn as nn
from sklearn.metrics import roc_auc_score
#from torchvision import datasets, transforms
#from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data.sampler import BatchSampler
import torch.nn.functional as F

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

In [2]:
print(device)

cuda


### For code optimization

In [3]:
# uncomment for code optimization
#torch.backends.cudnn.benchmark = True

NUM_WORKERS = int(os.cpu_count() / 2)
print('number of workers: ', NUM_WORKERS)

number of workers:  6


## 1. Helper Functions

### 1a. Evaluation (Loss, Eval Metrics)

In [4]:
# Loss Function
class HingeLoss(nn.Module):
    def __init__(self):
        super(HingeLoss, self).__init__()
    
    def forward(self, ypred, ytrue, margin = 1.0, smooth = False):
        ypred = ypred.squeeze()
        if smooth:
            loss = torch.nn.Softplus()
            out = torch.mean(loss(margin - (ytrue * ypred)))
        else:
            out = torch.mean(torch.relu(margin - (ytrue * ypred)))
        return out


def evalmetrics(y_true, scores):
    auc_score = roc_auc_score(y_true, scores)
    
    return auc_score

### 1b. Data Set Loaders

In [5]:
class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    # CODE SOURCE : https://discuss.pytorch.org/t/load-the-same-number-of-data-per-class/65198/4
    """
    def __init__(self, dataset, n_classes, n_samples, class_id = None, allSamples = False):
        loader = torch.utils.data.DataLoader(dataset)
        self.labels_list = []
        
        for _, label in loader:
            self.labels_list.append(label)
        
        self.labels = torch.LongTensor(self.labels_list)
        self.labels_set = list(set(self.labels.numpy()))
        print(self.labels_set)
        
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.dataset = dataset
        self.batch_size = self.n_samples * self.n_classes
        self.class_id = class_id
        self.allSamples = allSamples
 
    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            
            if self.class_id is not None:
                classes = self.class_id
            else:
                classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            
            indices = []
            for class_ in classes:
                if not self.allSamples: 
                    indices.extend(self.label_to_indices[class_][
                                   self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                        class_] + self.n_samples])
                else: 
                    indices.extend(self.label_to_indices[class_])
                
                self.used_label_indices_count[class_] += self.n_samples
                
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
                    
            yield indices
            
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return len(self.dataset) // self.batch_size

## 2. Models

In [6]:
class CIFAR10_LeNet(nn.Module):
    def __init__(self):
        super(CIFAR10_LeNet, self).__init__()

        self.rep_dim = 128
        self.pool = nn.MaxPool2d(2, 2)

        self.conv1 = nn.Conv2d(3, 32, 5, bias=False, padding=2)
        self.bn2d1 = nn.BatchNorm2d(32, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(32, 64, 5, bias=False, padding=2)
        self.bn2d2 = nn.BatchNorm2d(64, eps=1e-04, affine=False)
        self.conv3 = nn.Conv2d(64, 128, 5, bias=False, padding=2)
        self.bn2d3 = nn.BatchNorm2d(128, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(128 * 4 * 4, self.rep_dim, bias=False)
        self.fc2 = nn.Linear(self.rep_dim, int(self.rep_dim/2), bias=False)
        self.fc3 = nn.Linear(int(self.rep_dim/2), 1, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.leaky_relu(self.bn2d1(x)))
        x = self.conv2(x)
        x = self.pool(F.leaky_relu(self.bn2d2(x)))
        x = self.conv3(x)
        x = self.pool(F.leaky_relu(self.bn2d3(x)))
        x = x.view(x.size(0), -1)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 3. Training Schedule

In [7]:
logging.basicConfig(filename='Cifar10_DOC3.log', level=logging.INFO)

In [8]:
# number of repetitions per class
# Results reported in paper are the summary statistics over 10 repetitions (i.e., n_reps = 10)
n_reps = 1

### 3.0 Training data: class 0

In [9]:
EPOCH = 350 
BATCH_SIZEU = 256
BATCH_SIZE = 256
normalclass = 0

print('********************************')
print('Normal class: ')
print(normalclass)
logging.info('********************************')
logging.info('Normal class: ')
logging.info(normalclass)

print('lam = {},lr = {}, Cu = {}'.format(0.1,0.005,0.5))
logging.info('lam = {},lr = {}, Cu = {}'.format(0.1,0.005,0.5))


transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
for exp in range(n_reps):
    
    print('Exp No: {}'.format(exp))
    logging.info('Exp No: {}'.format(exp))

    ###############################################################################################
    #                                     Train Data
    ###############################################################################################
    cifar_train =  torchvision.datasets.CIFAR10(root="./cifar/cifar_train", train=True, download=True, 
                                              transform=transform)

    balanced_batch_sampler = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass])
    train_loader = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler, pin_memory=True, num_workers=NUM_WORKERS)

    
    ###############################################################################################
    #                                     Evaluations on :-
    ###############################################################################################
    # Test Data
    test_dat = torchvision.datasets.CIFAR10('./cifar', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dat, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    
    # Train (used to Evaluate i.e. TP_rate etc.)
    balanced_batch_sampler_eval = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass],allSamples=True)
    train_loader_eval = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler_eval, pin_memory=True, num_workers=NUM_WORKERS)

    
    lam = 0.1
    lr = 0.005
    Cu = 0.5
    
    
    eps = 0.0
    dpos = 1.0-eps
    dneg = -(1.0+eps)

    net = CIFAR10_LeNet()
    net = net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr = lr)  

    train_loss = HingeLoss()
    unlabeled_posloss = HingeLoss()
    unlabeled_negloss = HingeLoss()

    for epoch in range(EPOCH+1):

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()

        for (i,data) in enumerate(train_loader):

            # Training Data
            inputs, labels = data
            inputs = inputs.to(device)
            ytr = np.ones(len(inputs))
            ytr = torch.from_numpy(ytr).to(device).float()
            outputs = net(inputs) 

            XU = torch.from_numpy(np.random.randn(BATCH_SIZEU, 3, 32, 32))

            XU = XU.to(device).float()
            outputsU = net(XU)
            yunppos = np.ones(BATCH_SIZE)
            yunpneg = -np.ones(BATCH_SIZE)
            yupos = torch.from_numpy(yunppos).to(device).float()
            yuneg = torch.from_numpy(yunpneg).to(device).float()

            
            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Loss
            loss = train_loss(outputs,ytr,smooth = False)

            # Unlabeled data
            lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
            loss+=Cu*lossunlab

            lam = torch.tensor(lam).to(device)
            l2_reg = torch.tensor(0.).to(device)

            for name, param in net.named_parameters():
                l2_reg += torch.norm(param)**2

            loss += lam * l2_reg

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1


        if epoch%50==0:

            idx_label_score = []
            net.eval()
            with torch.no_grad():
                for data in test_loader:
                    inputs, labels = data

                    labels = labels.cpu().data.numpy()
                    labels = (labels==normalclass).astype(int)

                    inputs = inputs.to(device)
                    outputs = net(inputs)
                    scores = outputs.data.cpu().numpy()
                    idx_label_score += list(zip(labels.tolist(),
                                                scores.tolist()))

            tstlabels, scores = zip(*idx_label_score)
            tstlabels = np.array(tstlabels)

            tstlabels[np.where(tstlabels==0)]=-1.0
            scores = np.array(scores)

            Tst_auc_score = evalmetrics(tstlabels,scores.flatten())

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))

            print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))


    # Final Solution
    logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

    print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

********************************
Normal class: 
0
lam = 0.1,lr = 0.005, Cu = 0.5
Exp No: 0
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Epoch 0/350	 Time: 2.533	 Loss: 13.92626451 	 Test AUC :0.5304 
  Epoch 50/350	 Time: 1.632	 Loss: 2.08640998 	 Test AUC :0.6742 
  Epoch 100/350	 Time: 1.544	 Loss: 0.36120845 	 Test AUC :0.7239 
  Epoch 150/350	 Time: 1.548	 Loss: 0.10845619 	 Test AUC :0.7994 
  Epoch 200/350	 Time: 1.692	 Loss: 0.07070498 	 Test AUC :0.8087 
  Epoch 250/350	 Time: 1.630	 Loss: 0.06384046 	 Test AUC :0.8056 
  Epoch 300/350	 Time: 1.533	 Loss: 0.06316035 	 Test AUC :0.8099 
  Epoch 350/350	 Time: 1.598	 Loss: 0.06313583 	 Test AUC :0.8132 
 Final (TOT. EPOCHS 350)::  Loss: 0.06313583 	   AUC (Test) :0.8132 


### 3.1 Training data: class 1

In [10]:
EPOCH = 350 
BATCH_SIZEU = 256
BATCH_SIZE = 256
normalclass = 1

print('********************************')
print('Normal class: ')
print(normalclass)
logging.info('********************************')
logging.info('Normal class: ')
logging.info(normalclass)

print('lam = {},lr = {}, Cu = {}'.format(0.05,0.0005,0.1))
logging.info('lam = {},lr = {}, Cu = {}'.format(0.05,0.0005,0.1))


transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
for exp in range(n_reps):
    
    print('Exp No: {}'.format(exp))
    logging.info('Exp No: {}'.format(exp))

    ###############################################################################################
    #                                     Train Data
    ###############################################################################################
    cifar_train =  torchvision.datasets.CIFAR10(root="./cifar/cifar_train", train=True, download=True, 
                                              transform=transform)

    balanced_batch_sampler = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass])
    train_loader = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler, pin_memory=True, num_workers=NUM_WORKERS)

    
    ###############################################################################################
    #                                     Evaluations on :-
    ###############################################################################################
    # Test Data
    test_dat = torchvision.datasets.CIFAR10('./cifar', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dat, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    
    # Train (used to Evaluate i.e. TP_rate etc.)
    balanced_batch_sampler_eval = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass],allSamples=True)
    train_loader_eval = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler_eval, pin_memory=True, num_workers=NUM_WORKERS)

    
    lam = 0.05
    lr = 0.0005
    Cu = 0.1
    
    
    eps = 0.0
    dpos = 1.0-eps
    dneg = -(1.0+eps)

    net = CIFAR10_LeNet()
    net = net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr = lr)  

    train_loss = HingeLoss()
    unlabeled_posloss = HingeLoss()
    unlabeled_negloss = HingeLoss()

    for epoch in range(EPOCH+1):

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()

        for (i,data) in enumerate(train_loader):

            # Training Data
            inputs, labels = data
            inputs = inputs.to(device)
            ytr = np.ones(len(inputs))
            ytr = torch.from_numpy(ytr).to(device).float()
            outputs = net(inputs) 

            XU = torch.from_numpy(np.random.randn(BATCH_SIZEU, 3, 32, 32))

            XU = XU.to(device).float()
            outputsU = net(XU)
            yunppos = np.ones(BATCH_SIZE)
            yunpneg = -np.ones(BATCH_SIZE)
            yupos = torch.from_numpy(yunppos).to(device).float()
            yuneg = torch.from_numpy(yunpneg).to(device).float()

            
            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Loss
            loss = train_loss(outputs,ytr,smooth = False)

            # Unlabeled data
            lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
            loss+=Cu*lossunlab

            lam = torch.tensor(lam).to(device)
            l2_reg = torch.tensor(0.).to(device)

            for name, param in net.named_parameters():
                l2_reg += torch.norm(param)**2

            loss += lam * l2_reg

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1


        if epoch%50==0:

            idx_label_score = []
            net.eval()
            with torch.no_grad():
                for data in test_loader:
                    inputs, labels = data

                    labels = labels.cpu().data.numpy()
                    labels = (labels==normalclass).astype(int)

                    inputs = inputs.to(device)
                    outputs = net(inputs)
                    scores = outputs.data.cpu().numpy()
                    idx_label_score += list(zip(labels.tolist(),
                                                scores.tolist()))

            tstlabels, scores = zip(*idx_label_score)
            tstlabels = np.array(tstlabels)

            tstlabels[np.where(tstlabels==0)]=-1.0
            scores = np.array(scores)

            Tst_auc_score = evalmetrics(tstlabels,scores.flatten())

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))

            print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))


    # Final Solution
    logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

    print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

********************************
Normal class: 
1
lam = 0.05,lr = 0.0005, Cu = 0.1
Exp No: 0
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Epoch 0/350	 Time: 1.632	 Loss: 7.70551486 	 Test AUC :0.6038 
  Epoch 50/350	 Time: 1.698	 Loss: 6.30869429 	 Test AUC :0.7135 
  Epoch 100/350	 Time: 1.676	 Loss: 5.73662268 	 Test AUC :0.7233 
  Epoch 150/350	 Time: 1.651	 Loss: 5.21678822 	 Test AUC :0.7278 
  Epoch 200/350	 Time: 1.725	 Loss: 4.74436348 	 Test AUC :0.7270 
  Epoch 250/350	 Time: 1.611	 Loss: 4.31490803 	 Test AUC :0.7290 
  Epoch 300/350	 Time: 1.609	 Loss: 3.92443804 	 Test AUC :0.7320 
  Epoch 350/350	 Time: 1.643	 Loss: 3.56951341 	 Test AUC :0.7344 
 Final (TOT. EPOCHS 350)::  Loss: 3.56951341 	   AUC (Test) :0.7344 


### 3.2 Training data: class 2

In [11]:
EPOCH = 350 
BATCH_SIZEU = 256
BATCH_SIZE = 256
normalclass = 2

print('********************************')
print('Normal class: ')
print(normalclass)
logging.info('********************************')
logging.info('Normal class: ')
logging.info(normalclass)

print('lam = {},lr = {}, Cu = {}'.format(0.05,0.005,0.5))
logging.info('lam = {},lr = {}, Cu = {}'.format(0.05,0.005,0.5))


transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
for exp in range(n_reps):
    
    print('Exp No: {}'.format(exp))
    logging.info('Exp No: {}'.format(exp))

    ###############################################################################################
    #                                     Train Data
    ###############################################################################################
    cifar_train =  torchvision.datasets.CIFAR10(root="./cifar/cifar_train", train=True, download=True, 
                                              transform=transform)

    balanced_batch_sampler = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass])
    train_loader = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler, pin_memory=True, num_workers=NUM_WORKERS)

    
    ###############################################################################################
    #                                     Evaluations on :-
    ###############################################################################################
    # Test Data
    test_dat = torchvision.datasets.CIFAR10('./cifar', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dat, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    
    # Train (used to Evaluate i.e. TP_rate etc.)
    balanced_batch_sampler_eval = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass],allSamples=True)
    train_loader_eval = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler_eval, pin_memory=True, num_workers=NUM_WORKERS)

    
    lam = 0.05
    lr = 0.005
    Cu = 0.5
    
    
    eps = 0.0
    dpos = 1.0-eps
    dneg = -(1.0+eps)

    net = CIFAR10_LeNet()
    net = net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr = lr)  

    train_loss = HingeLoss()
    unlabeled_posloss = HingeLoss()
    unlabeled_negloss = HingeLoss()

    for epoch in range(EPOCH+1):

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()

        for (i,data) in enumerate(train_loader):

            # Training Data
            inputs, labels = data
            inputs = inputs.to(device)
            ytr = np.ones(len(inputs))
            ytr = torch.from_numpy(ytr).to(device).float()
            outputs = net(inputs) 

            XU = torch.from_numpy(np.random.randn(BATCH_SIZEU, 3, 32, 32))

            XU = XU.to(device).float()
            outputsU = net(XU)
            yunppos = np.ones(BATCH_SIZE)
            yunpneg = -np.ones(BATCH_SIZE)
            yupos = torch.from_numpy(yunppos).to(device).float()
            yuneg = torch.from_numpy(yunpneg).to(device).float()

            
            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Loss
            loss = train_loss(outputs,ytr,smooth = False)

            # Unlabeled data
            lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
            loss+=Cu*lossunlab

            lam = torch.tensor(lam).to(device)
            l2_reg = torch.tensor(0.).to(device)

            for name, param in net.named_parameters():
                l2_reg += torch.norm(param)**2

            loss += lam * l2_reg

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1


        if epoch%50==0:

            idx_label_score = []
            net.eval()
            with torch.no_grad():
                for data in test_loader:
                    inputs, labels = data

                    labels = labels.cpu().data.numpy()
                    labels = (labels==normalclass).astype(int)

                    inputs = inputs.to(device)
                    outputs = net(inputs)
                    scores = outputs.data.cpu().numpy()
                    idx_label_score += list(zip(labels.tolist(),
                                                scores.tolist()))

            tstlabels, scores = zip(*idx_label_score)
            tstlabels = np.array(tstlabels)

            tstlabels[np.where(tstlabels==0)]=-1.0
            scores = np.array(scores)

            Tst_auc_score = evalmetrics(tstlabels,scores.flatten())

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))

            print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))


    # Final Solution
    logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

    print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

********************************
Normal class: 
2
lam = 0.05,lr = 0.005, Cu = 0.5
Exp No: 0
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Epoch 0/350	 Time: 1.785	 Loss: 7.12907111 	 Test AUC :0.5095 
  Epoch 50/350	 Time: 1.694	 Loss: 2.68724964 	 Test AUC :0.6051 
  Epoch 100/350	 Time: 1.685	 Loss: 1.06182805 	 Test AUC :0.6378 
  Epoch 150/350	 Time: 1.648	 Loss: 0.43700445 	 Test AUC :0.6418 
  Epoch 200/350	 Time: 1.729	 Loss: 0.19621352 	 Test AUC :0.6544 
  Epoch 250/350	 Time: 1.669	 Loss: 0.10839283 	 Test AUC :0.6820 
  Epoch 300/350	 Time: 1.675	 Loss: 0.06906865 	 Test AUC :0.6881 
  Epoch 350/350	 Time: 1.688	 Loss: 0.05747793 	 Test AUC :0.6870 
 Final (TOT. EPOCHS 350)::  Loss: 0.05747793 	   AUC (Test) :0.6870 


### 3.3 Training data: class 3

In [12]:
EPOCH = 350 
BATCH_SIZEU = 256
BATCH_SIZE = 256
normalclass = 3

print('********************************')
print('Normal class: ')
print(normalclass)
logging.info('********************************')
logging.info('Normal class: ')
logging.info(normalclass)

print('lam = {},lr = {}, Cu = {}'.format(0.1,0.005,1.0))
logging.info('lam = {},lr = {}, Cu = {}'.format(0.1,0.005,1.0))


transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
for exp in range(n_reps):
    
    print('Exp No: {}'.format(exp))
    logging.info('Exp No: {}'.format(exp))

    ###############################################################################################
    #                                     Train Data
    ###############################################################################################
    cifar_train =  torchvision.datasets.CIFAR10(root="./cifar/cifar_train", train=True, download=True, 
                                              transform=transform)

    balanced_batch_sampler = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass])
    train_loader = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler, pin_memory=True, num_workers=NUM_WORKERS)

    
    ###############################################################################################
    #                                     Evaluations on :-
    ###############################################################################################
    # Test Data
    test_dat = torchvision.datasets.CIFAR10('./cifar', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dat, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    
    # Train (used to Evaluate i.e. TP_rate etc.)
    balanced_batch_sampler_eval = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass],allSamples=True)
    train_loader_eval = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler_eval, pin_memory=True, num_workers=NUM_WORKERS)

    
    lam = 0.1
    lr = 0.005
    Cu = 1.0
    
    
    eps = 0.0
    dpos = 1.0-eps
    dneg = -(1.0+eps)

    net = CIFAR10_LeNet()
    net = net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr = lr)  

    train_loss = HingeLoss()
    unlabeled_posloss = HingeLoss()
    unlabeled_negloss = HingeLoss()

    for epoch in range(EPOCH+1):

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()

        for (i,data) in enumerate(train_loader):

            # Training Data
            inputs, labels = data
            inputs = inputs.to(device)
            ytr = np.ones(len(inputs))
            ytr = torch.from_numpy(ytr).to(device).float()
            outputs = net(inputs) 

            XU = torch.from_numpy(np.random.randn(BATCH_SIZEU, 3, 32, 32))

            XU = XU.to(device).float()
            outputsU = net(XU)
            yunppos = np.ones(BATCH_SIZE)
            yunpneg = -np.ones(BATCH_SIZE)
            yupos = torch.from_numpy(yunppos).to(device).float()
            yuneg = torch.from_numpy(yunpneg).to(device).float()

            
            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Loss
            loss = train_loss(outputs,ytr,smooth = False)

            # Unlabeled data
            lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
            loss+=Cu*lossunlab

            lam = torch.tensor(lam).to(device)
            l2_reg = torch.tensor(0.).to(device)

            for name, param in net.named_parameters():
                l2_reg += torch.norm(param)**2

            loss += lam * l2_reg

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1


        if epoch%50==0:

            idx_label_score = []
            net.eval()
            with torch.no_grad():
                for data in test_loader:
                    inputs, labels = data

                    labels = labels.cpu().data.numpy()
                    labels = (labels==normalclass).astype(int)

                    inputs = inputs.to(device)
                    outputs = net(inputs)
                    scores = outputs.data.cpu().numpy()
                    idx_label_score += list(zip(labels.tolist(),
                                                scores.tolist()))

            tstlabels, scores = zip(*idx_label_score)
            tstlabels = np.array(tstlabels)

            tstlabels[np.where(tstlabels==0)]=-1.0
            scores = np.array(scores)

            Tst_auc_score = evalmetrics(tstlabels,scores.flatten())

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))

            print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))


    # Final Solution
    logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

    print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

********************************
Normal class: 
3
lam = 0.1,lr = 0.005, Cu = 1.0
Exp No: 0
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Epoch 0/350	 Time: 1.842	 Loss: 14.13794849 	 Test AUC :0.4979 
  Epoch 50/350	 Time: 1.765	 Loss: 2.14326912 	 Test AUC :0.5305 
  Epoch 100/350	 Time: 1.718	 Loss: 0.43353922 	 Test AUC :0.5834 
  Epoch 150/350	 Time: 1.798	 Loss: 0.18824741 	 Test AUC :0.6255 
  Epoch 200/350	 Time: 1.700	 Loss: 0.15213570 	 Test AUC :0.6284 
  Epoch 250/350	 Time: 1.689	 Loss: 0.14192049 	 Test AUC :0.6278 
  Epoch 300/350	 Time: 1.655	 Loss: 0.14947615 	 Test AUC :0.6263 
  Epoch 350/350	 Time: 1.798	 Loss: 0.14925644 	 Test AUC :0.6247 
 Final (TOT. EPOCHS 350)::  Loss: 0.14925644 	   AUC (Test) :0.6247 


### 3.4 Training data: class 4

In [13]:
EPOCH = 350 
BATCH_SIZEU = 256
BATCH_SIZE = 256
normalclass = 4

print('********************************')
print('Normal class: ')
print(normalclass)
logging.info('********************************')
logging.info('Normal class: ')
logging.info(normalclass)

print('lam = {},lr = {}, Cu = {}'.format(0.1,0.001,1.0))
logging.info('lam = {},lr = {}, Cu = {}'.format(0.1,0.001,1.0))


transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
for exp in range(n_reps):
    
    print('Exp No: {}'.format(exp))
    logging.info('Exp No: {}'.format(exp))

    ###############################################################################################
    #                                     Train Data
    ###############################################################################################
    cifar_train =  torchvision.datasets.CIFAR10(root="./cifar/cifar_train", train=True, download=True, 
                                              transform=transform)

    balanced_batch_sampler = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass])
    train_loader = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler, pin_memory=True, num_workers=NUM_WORKERS)

    
    ###############################################################################################
    #                                     Evaluations on :-
    ###############################################################################################
    # Test Data
    test_dat = torchvision.datasets.CIFAR10('./cifar', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dat, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    
    # Train (used to Evaluate i.e. TP_rate etc.)
    balanced_batch_sampler_eval = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass],allSamples=True)
    train_loader_eval = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler_eval, pin_memory=True, num_workers=NUM_WORKERS)

    
    lam = 0.1
    lr = 0.001
    Cu = 1.0
    
    
    eps = 0.0
    dpos = 1.0-eps
    dneg = -(1.0+eps)

    net = CIFAR10_LeNet()
    net = net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr = lr)  

    train_loss = HingeLoss()
    unlabeled_posloss = HingeLoss()
    unlabeled_negloss = HingeLoss()

    for epoch in range(EPOCH+1):

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()

        for (i,data) in enumerate(train_loader):

            # Training Data
            inputs, labels = data
            inputs = inputs.to(device)
            ytr = np.ones(len(inputs))
            ytr = torch.from_numpy(ytr).to(device).float()
            outputs = net(inputs) 

            XU = torch.from_numpy(np.random.randn(BATCH_SIZEU, 3, 32, 32))

            XU = XU.to(device).float()
            outputsU = net(XU)
            yunppos = np.ones(BATCH_SIZE)
            yunpneg = -np.ones(BATCH_SIZE)
            yupos = torch.from_numpy(yunppos).to(device).float()
            yuneg = torch.from_numpy(yunpneg).to(device).float()

            
            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Loss
            loss = train_loss(outputs,ytr,smooth = False)

            # Unlabeled data
            lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
            loss+=Cu*lossunlab

            lam = torch.tensor(lam).to(device)
            l2_reg = torch.tensor(0.).to(device)

            for name, param in net.named_parameters():
                l2_reg += torch.norm(param)**2

            loss += lam * l2_reg

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1


        if epoch%50==0:

            idx_label_score = []
            net.eval()
            with torch.no_grad():
                for data in test_loader:
                    inputs, labels = data

                    labels = labels.cpu().data.numpy()
                    labels = (labels==normalclass).astype(int)

                    inputs = inputs.to(device)
                    outputs = net(inputs)
                    scores = outputs.data.cpu().numpy()
                    idx_label_score += list(zip(labels.tolist(),
                                                scores.tolist()))

            tstlabels, scores = zip(*idx_label_score)
            tstlabels = np.array(tstlabels)

            tstlabels[np.where(tstlabels==0)]=-1.0
            scores = np.array(scores)

            Tst_auc_score = evalmetrics(tstlabels,scores.flatten())

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))

            print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))


    # Final Solution
    logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

    print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

********************************
Normal class: 
4
lam = 0.1,lr = 0.001, Cu = 1.0
Exp No: 0
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Epoch 0/350	 Time: 1.705	 Loss: 14.61869782 	 Test AUC :0.3067 
  Epoch 50/350	 Time: 1.643	 Loss: 9.45128857 	 Test AUC :0.4861 
  Epoch 100/350	 Time: 1.632	 Loss: 6.47634697 	 Test AUC :0.5483 
  Epoch 150/350	 Time: 1.779	 Loss: 4.44396313 	 Test AUC :0.6136 
  Epoch 200/350	 Time: 1.646	 Loss: 3.05551900 	 Test AUC :0.6659 
  Epoch 250/350	 Time: 1.670	 Loss: 2.10612060 	 Test AUC :0.6978 
  Epoch 300/350	 Time: 1.655	 Loss: 1.45633841 	 Test AUC :0.7161 
  Epoch 350/350	 Time: 1.746	 Loss: 1.01245551 	 Test AUC :0.7271 
 Final (TOT. EPOCHS 350)::  Loss: 1.01245551 	   AUC (Test) :0.7271 


### 3.5 Training data: class 5

In [14]:
EPOCH = 350 
BATCH_SIZEU = 256
BATCH_SIZE = 256
normalclass = 5

print('********************************')
print('Normal class: ')
print(normalclass)
logging.info('********************************')
logging.info('Normal class: ')
logging.info(normalclass)

print('lam = {},lr = {}, Cu = {}'.format(0.05,0.001,1.0))
logging.info('lam = {},lr = {}, Cu = {}'.format(0.05,0.001,1.0))

tst_score = []

transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
for exp in range(n_reps):
    
    print('Exp No: {}'.format(exp))
    logging.info('Exp No: {}'.format(exp))

    ###############################################################################################
    #                                     Train Data
    ###############################################################################################
    cifar_train =  torchvision.datasets.CIFAR10(root="./cifar/cifar_train", train=True, download=True, 
                                              transform=transform)

    balanced_batch_sampler = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass])
    train_loader = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler, pin_memory=True, num_workers=NUM_WORKERS)

    
    ###############################################################################################
    #                                     Evaluations on :-
    ###############################################################################################
    # Test Data
    test_dat = torchvision.datasets.CIFAR10('./cifar', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dat, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    
    # Train (used to Evaluate i.e. TP_rate etc.)
    balanced_batch_sampler_eval = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass],allSamples=True)
    train_loader_eval = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler_eval, pin_memory=True, num_workers=NUM_WORKERS)

    
    lam = 0.05
    lr = 0.001
    Cu = 1.0
    
    
    eps = 0.0
    dpos = 1.0-eps
    dneg = -(1.0+eps)

    net = CIFAR10_LeNet()
    net = net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr = lr)  

    train_loss = HingeLoss()
    unlabeled_posloss = HingeLoss()
    unlabeled_negloss = HingeLoss()

    for epoch in range(EPOCH+1):

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()

        for (i,data) in enumerate(train_loader):

            # Training Data
            inputs, labels = data
            inputs = inputs.to(device)
            ytr = np.ones(len(inputs))
            ytr = torch.from_numpy(ytr).to(device).float()
            outputs = net(inputs) 

            XU = torch.from_numpy(np.random.randn(BATCH_SIZEU, 3, 32, 32))

            XU = XU.to(device).float()
            outputsU = net(XU)
            yunppos = np.ones(BATCH_SIZE)
            yunpneg = -np.ones(BATCH_SIZE)
            yupos = torch.from_numpy(yunppos).to(device).float()
            yuneg = torch.from_numpy(yunpneg).to(device).float()

            
            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Loss
            loss = train_loss(outputs,ytr,smooth = False)

            # Unlabeled data
            lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
            loss+=Cu*lossunlab

            lam = torch.tensor(lam).to(device)
            l2_reg = torch.tensor(0.).to(device)

            for name, param in net.named_parameters():
                l2_reg += torch.norm(param)**2

            loss += lam * l2_reg

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1


        if epoch%50==0:

            idx_label_score = []
            net.eval()
            with torch.no_grad():
                for data in test_loader:
                    inputs, labels = data

                    labels = labels.cpu().data.numpy()
                    labels = (labels==normalclass).astype(int)

                    inputs = inputs.to(device)
                    outputs = net(inputs)
                    scores = outputs.data.cpu().numpy()
                    idx_label_score += list(zip(labels.tolist(),
                                                scores.tolist()))

            tstlabels, scores = zip(*idx_label_score)
            tstlabels = np.array(tstlabels)

            tstlabels[np.where(tstlabels==0)]=-1.0
            scores = np.array(scores)

            Tst_auc_score = evalmetrics(tstlabels,scores.flatten())

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))

            print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))


    # Final Solution
    logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

    print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

********************************
Normal class: 
5
lam = 0.05,lr = 0.001, Cu = 1.0
Exp No: 0
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Epoch 0/350	 Time: 1.735	 Loss: 7.65478498 	 Test AUC :0.5589 
  Epoch 50/350	 Time: 1.705	 Loss: 5.76606043 	 Test AUC :0.6440 
  Epoch 100/350	 Time: 1.705	 Loss: 4.76864526 	 Test AUC :0.6528 
  Epoch 150/350	 Time: 1.752	 Loss: 3.94667141 	 Test AUC :0.6525 
  Epoch 200/350	 Time: 1.664	 Loss: 3.26845138 	 Test AUC :0.6537 
  Epoch 250/350	 Time: 1.608	 Loss: 2.70823157 	 Test AUC :0.6492 
  Epoch 300/350	 Time: 1.671	 Loss: 2.24520887 	 Test AUC :0.6396 
  Epoch 350/350	 Time: 1.843	 Loss: 1.86254630 	 Test AUC :0.6209 
 Final (TOT. EPOCHS 350)::  Loss: 1.86254630 	   AUC (Test) :0.6209 


### 3.6 Training data: class 6

In [15]:
EPOCH = 350 
BATCH_SIZEU = 256
BATCH_SIZE = 256
normalclass = 6

print('********************************')
print('Normal class: ')
print(normalclass)
logging.info('********************************')
logging.info('Normal class: ')
logging.info(normalclass)

print('lam = {},lr = {}, Cu = {}'.format(0.1,0.005,0.5))
logging.info('lam = {},lr = {}, Cu = {}'.format(0.1,0.005,0.5))


transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
for exp in range(n_reps):
    
    print('Exp No: {}'.format(exp))
    logging.info('Exp No: {}'.format(exp))

    ###############################################################################################
    #                                     Train Data
    ###############################################################################################
    cifar_train =  torchvision.datasets.CIFAR10(root="./cifar/cifar_train", train=True, download=True, 
                                              transform=transform)

    balanced_batch_sampler = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass])
    train_loader = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler, pin_memory=True, num_workers=NUM_WORKERS)

    
    ###############################################################################################
    #                                     Evaluations on :-
    ###############################################################################################
    # Test Data
    test_dat = torchvision.datasets.CIFAR10('./cifar', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dat, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    
    # Train (used to Evaluate i.e. TP_rate etc.)
    balanced_batch_sampler_eval = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass],allSamples=True)
    train_loader_eval = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler_eval, pin_memory=True, num_workers=NUM_WORKERS)

    
    lam = 0.1
    lr = 0.005
    Cu = 0.5
    
    
    eps = 0.0
    dpos = 1.0-eps
    dneg = -(1.0+eps)

    net = CIFAR10_LeNet()
    net = net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr = lr)  

    train_loss = HingeLoss()
    unlabeled_posloss = HingeLoss()
    unlabeled_negloss = HingeLoss()

    for epoch in range(EPOCH+1):

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()

        for (i,data) in enumerate(train_loader):

            # Training Data
            inputs, labels = data
            inputs = inputs.to(device)
            ytr = np.ones(len(inputs))
            ytr = torch.from_numpy(ytr).to(device).float()
            outputs = net(inputs) 

            XU = torch.from_numpy(np.random.randn(BATCH_SIZEU, 3, 32, 32))

            XU = XU.to(device).float()
            outputsU = net(XU)
            yunppos = np.ones(BATCH_SIZE)
            yunpneg = -np.ones(BATCH_SIZE)
            yupos = torch.from_numpy(yunppos).to(device).float()
            yuneg = torch.from_numpy(yunpneg).to(device).float()

            
            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Loss
            loss = train_loss(outputs,ytr,smooth = False)

            # Unlabeled data
            lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
            loss+=Cu*lossunlab

            lam = torch.tensor(lam).to(device)
            l2_reg = torch.tensor(0.).to(device)

            for name, param in net.named_parameters():
                l2_reg += torch.norm(param)**2

            loss += lam * l2_reg

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1


        if epoch%50==0:

            idx_label_score = []
            net.eval()
            with torch.no_grad():
                for data in test_loader:
                    inputs, labels = data

                    labels = labels.cpu().data.numpy()
                    labels = (labels==normalclass).astype(int)

                    inputs = inputs.to(device)
                    outputs = net(inputs)
                    scores = outputs.data.cpu().numpy()
                    idx_label_score += list(zip(labels.tolist(),
                                                scores.tolist()))

            tstlabels, scores = zip(*idx_label_score)
            tstlabels = np.array(tstlabels)

            tstlabels[np.where(tstlabels==0)]=-1.0
            scores = np.array(scores)

            Tst_auc_score = evalmetrics(tstlabels,scores.flatten())

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))

            print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))


    # Final Solution
    logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

    print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

********************************
Normal class: 
6
lam = 0.1,lr = 0.005, Cu = 0.5
Exp No: 0
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Epoch 0/350	 Time: 1.847	 Loss: 14.01391662 	 Test AUC :0.4667 
  Epoch 50/350	 Time: 1.695	 Loss: 2.09322165 	 Test AUC :0.7159 
  Epoch 100/350	 Time: 1.685	 Loss: 0.36160782 	 Test AUC :0.7269 
  Epoch 150/350	 Time: 1.836	 Loss: 0.10982294 	 Test AUC :0.7475 
  Epoch 200/350	 Time: 1.684	 Loss: 0.07933822 	 Test AUC :0.7557 
  Epoch 250/350	 Time: 1.690	 Loss: 0.06756092 	 Test AUC :0.7619 
  Epoch 300/350	 Time: 1.698	 Loss: 0.06784672 	 Test AUC :0.7605 
  Epoch 350/350	 Time: 1.763	 Loss: 0.06705865 	 Test AUC :0.7641 
 Final (TOT. EPOCHS 350)::  Loss: 0.06705865 	   AUC (Test) :0.7641 


### 3.7 Training data: class 7

In [16]:
EPOCH = 350 
BATCH_SIZEU = 256
BATCH_SIZE = 256
normalclass = 7

print('********************************')
print('Normal class: ')
print(normalclass)
logging.info('********************************')
logging.info('Normal class: ')
logging.info(normalclass)

print('lam = {},lr = {}, Cu = {}'.format(0.05,0.0005,1.0))
logging.info('lam = {},lr = {}, Cu = {}'.format(0.05,0.0005,1.0))


transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
for exp in range(n_reps):
    
    print('Exp No: {}'.format(exp))
    logging.info('Exp No: {}'.format(exp))

    ###############################################################################################
    #                                     Train Data
    ###############################################################################################
    cifar_train =  torchvision.datasets.CIFAR10(root="./cifar/cifar_train", train=True, download=True, 
                                              transform=transform)

    balanced_batch_sampler = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass])
    train_loader = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler, pin_memory=True, num_workers=NUM_WORKERS)

    
    ###############################################################################################
    #                                     Evaluations on :-
    ###############################################################################################
    # Test Data
    test_dat = torchvision.datasets.CIFAR10('./cifar', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dat, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    
    # Train (used to Evaluate i.e. TP_rate etc.)
    balanced_batch_sampler_eval = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass],allSamples=True)
    train_loader_eval = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler_eval, pin_memory=True, num_workers=NUM_WORKERS)

    
    lam = 0.05
    lr = 0.0005
    Cu = 1.0
    
    
    eps = 0.0
    dpos = 1.0-eps
    dneg = -(1.0+eps)

    net = CIFAR10_LeNet()
    net = net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr = lr)  

    train_loss = HingeLoss()
    unlabeled_posloss = HingeLoss()
    unlabeled_negloss = HingeLoss()

    for epoch in range(EPOCH+1):

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()

        for (i,data) in enumerate(train_loader):

            # Training Data
            inputs, labels = data
            inputs = inputs.to(device)
            ytr = np.ones(len(inputs))
            ytr = torch.from_numpy(ytr).to(device).float()
            outputs = net(inputs) 

            XU = torch.from_numpy(np.random.randn(BATCH_SIZEU, 3, 32, 32))

            XU = XU.to(device).float()
            outputsU = net(XU)
            yunppos = np.ones(BATCH_SIZE)
            yunpneg = -np.ones(BATCH_SIZE)
            yupos = torch.from_numpy(yunppos).to(device).float()
            yuneg = torch.from_numpy(yunpneg).to(device).float()

            
            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Loss
            loss = train_loss(outputs,ytr,smooth = False)

            # Unlabeled data
            lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
            loss+=Cu*lossunlab

            lam = torch.tensor(lam).to(device)
            l2_reg = torch.tensor(0.).to(device)

            for name, param in net.named_parameters():
                l2_reg += torch.norm(param)**2

            loss += lam * l2_reg

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1


        if epoch%50==0:

            idx_label_score = []
            net.eval()
            with torch.no_grad():
                for data in test_loader:
                    inputs, labels = data

                    labels = labels.cpu().data.numpy()
                    labels = (labels==normalclass).astype(int)

                    inputs = inputs.to(device)
                    outputs = net(inputs)
                    scores = outputs.data.cpu().numpy()
                    idx_label_score += list(zip(labels.tolist(),
                                                scores.tolist()))

            tstlabels, scores = zip(*idx_label_score)
            tstlabels = np.array(tstlabels)

            tstlabels[np.where(tstlabels==0)]=-1.0
            scores = np.array(scores)

            Tst_auc_score = evalmetrics(tstlabels,scores.flatten())

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))

            print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))


    # Final Solution
    logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

    print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

********************************
Normal class: 
7
lam = 0.05,lr = 0.0005, Cu = 1.0
Exp No: 0
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Epoch 0/350	 Time: 1.730	 Loss: 8.35768243 	 Test AUC :0.6058 
  Epoch 50/350	 Time: 1.688	 Loss: 6.36298385 	 Test AUC :0.6633 
  Epoch 100/350	 Time: 1.664	 Loss: 5.78328231 	 Test AUC :0.6555 
  Epoch 150/350	 Time: 1.847	 Loss: 5.26072843 	 Test AUC :0.6501 
  Epoch 200/350	 Time: 1.654	 Loss: 4.78545984 	 Test AUC :0.6482 
  Epoch 250/350	 Time: 1.688	 Loss: 4.35467921 	 Test AUC :0.6496 
  Epoch 300/350	 Time: 1.684	 Loss: 3.96300528 	 Test AUC :0.6515 
  Epoch 350/350	 Time: 1.779	 Loss: 3.60627202 	 Test AUC :0.6533 
 Final (TOT. EPOCHS 350)::  Loss: 3.60627202 	   AUC (Test) :0.6533 


### 3.8 Training data: class 8

In [17]:
EPOCH = 350 
BATCH_SIZEU = 256
BATCH_SIZE = 256
normalclass = 8

print('********************************')
print('Normal class: ')
print(normalclass)
logging.info('********************************')
logging.info('Normal class: ')
logging.info(normalclass)

print('lam = {},lr = {}, Cu = {}'.format(0.05,0.005,0.1))
logging.info('lam = {},lr = {}, Cu = {}'.format(0.05,0.005,0.1))


transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
for exp in range(n_reps):
    
    print('Exp No: {}'.format(exp))
    logging.info('Exp No: {}'.format(exp))

    ###############################################################################################
    #                                     Train Data
    ###############################################################################################
    cifar_train =  torchvision.datasets.CIFAR10(root="./cifar/cifar_train", train=True, download=True, 
                                              transform=transform)

    balanced_batch_sampler = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass])
    train_loader = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler, pin_memory=True, num_workers=NUM_WORKERS)

    
    ###############################################################################################
    #                                     Evaluations on :-
    ###############################################################################################
    # Test Data
    test_dat = torchvision.datasets.CIFAR10('./cifar', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dat, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    
    # Train (used to Evaluate i.e. TP_rate etc.)
    balanced_batch_sampler_eval = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass],allSamples=True)
    train_loader_eval = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler_eval, pin_memory=True, num_workers=NUM_WORKERS)

    
    lam = 0.05
    lr = 0.005
    Cu = 0.1
    
    
    eps = 0.0
    dpos = 1.0-eps
    dneg = -(1.0+eps)

    net = CIFAR10_LeNet()
    net = net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr = lr)  

    train_loss = HingeLoss()
    unlabeled_posloss = HingeLoss()
    unlabeled_negloss = HingeLoss()

    for epoch in range(EPOCH+1):

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()

        for (i,data) in enumerate(train_loader):

            # Training Data
            inputs, labels = data
            inputs = inputs.to(device)
            ytr = np.ones(len(inputs))
            ytr = torch.from_numpy(ytr).to(device).float()
            outputs = net(inputs) 

            XU = torch.from_numpy(np.random.randn(BATCH_SIZEU, 3, 32, 32))

            XU = XU.to(device).float()
            outputsU = net(XU)
            yunppos = np.ones(BATCH_SIZE)
            yunpneg = -np.ones(BATCH_SIZE)
            yupos = torch.from_numpy(yunppos).to(device).float()
            yuneg = torch.from_numpy(yunpneg).to(device).float()

            
            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Loss
            loss = train_loss(outputs,ytr,smooth = False)

            # Unlabeled data
            lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
            loss+=Cu*lossunlab

            lam = torch.tensor(lam).to(device)
            l2_reg = torch.tensor(0.).to(device)

            for name, param in net.named_parameters():
                l2_reg += torch.norm(param)**2

            loss += lam * l2_reg

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1


        if epoch%50==0:

            idx_label_score = []
            net.eval()
            with torch.no_grad():
                for data in test_loader:
                    inputs, labels = data

                    labels = labels.cpu().data.numpy()
                    labels = (labels==normalclass).astype(int)

                    inputs = inputs.to(device)
                    outputs = net(inputs)
                    scores = outputs.data.cpu().numpy()
                    idx_label_score += list(zip(labels.tolist(),
                                                scores.tolist()))

            tstlabels, scores = zip(*idx_label_score)
            tstlabels = np.array(tstlabels)

            tstlabels[np.where(tstlabels==0)]=-1.0
            scores = np.array(scores)

            Tst_auc_score = evalmetrics(tstlabels,scores.flatten())

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))

            print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))


    # Final Solution
    logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

    print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

********************************
Normal class: 
8
lam = 0.05,lr = 0.005, Cu = 0.1
Exp No: 0
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Epoch 0/350	 Time: 1.784	 Loss: 7.03883036 	 Test AUC :0.6119 
  Epoch 50/350	 Time: 1.674	 Loss: 2.67228883 	 Test AUC :0.5226 
  Epoch 100/350	 Time: 1.654	 Loss: 1.04039698 	 Test AUC :0.5581 
  Epoch 150/350	 Time: 1.750	 Loss: 0.40995010 	 Test AUC :0.6883 
  Epoch 200/350	 Time: 1.713	 Loss: 0.16630792 	 Test AUC :0.7148 
  Epoch 250/350	 Time: 1.641	 Loss: 0.07294629 	 Test AUC :0.7331 
  Epoch 300/350	 Time: 1.798	 Loss: 0.03935403 	 Test AUC :0.7729 
  Epoch 350/350	 Time: 1.631	 Loss: 0.02281585 	 Test AUC :0.8104 
 Final (TOT. EPOCHS 350)::  Loss: 0.02281585 	   AUC (Test) :0.8104 


### 3.9 Training data: class 9

In [18]:
EPOCH = 350 
BATCH_SIZEU = 256
BATCH_SIZE = 256
normalclass = 9

print('********************************')
print('Normal class: ')
print(normalclass)
logging.info('********************************')
logging.info('Normal class: ')
logging.info(normalclass)

print('lam = {},lr = {}, Cu = {}'.format(0.05,0.0001,0.05))
logging.info('lam = {},lr = {}, Cu = {}'.format(0.05,0.0001,0.05))


transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
for exp in range(n_reps):
    
    print('Exp No: {}'.format(exp))
    logging.info('Exp No: {}'.format(exp))

    ###############################################################################################
    #                                     Train Data
    ###############################################################################################
    cifar_train =  torchvision.datasets.CIFAR10(root="./cifar/cifar_train", train=True, download=True, 
                                              transform=transform)

    balanced_batch_sampler = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass])
    train_loader = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler, pin_memory=True, num_workers=NUM_WORKERS)

    
    ###############################################################################################
    #                                     Evaluations on :-
    ###############################################################################################
    # Test Data
    test_dat = torchvision.datasets.CIFAR10('./cifar', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dat, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    
    # Train (used to Evaluate i.e. TP_rate etc.)
    balanced_batch_sampler_eval = BalancedBatchSampler(cifar_train, 10, n_samples = BATCH_SIZE, class_id = [normalclass],allSamples=True)
    train_loader_eval = torch.utils.data.DataLoader(cifar_train, batch_sampler=balanced_batch_sampler_eval, pin_memory=True, num_workers=NUM_WORKERS)

    
    lam = 0.05
    lr = 0.0001
    Cu = 0.05
    
    
    eps = 0.0
    dpos = 1.0-eps
    dneg = -(1.0+eps)

    net = CIFAR10_LeNet()
    net = net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr = lr)  

    train_loss = HingeLoss()
    unlabeled_posloss = HingeLoss()
    unlabeled_negloss = HingeLoss()

    for epoch in range(EPOCH+1):

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()

        for (i,data) in enumerate(train_loader):

            # Training Data
            inputs, labels = data
            inputs = inputs.to(device)
            ytr = np.ones(len(inputs))
            ytr = torch.from_numpy(ytr).to(device).float()
            outputs = net(inputs) 

            XU = torch.from_numpy(np.random.randn(BATCH_SIZEU, 3, 32, 32))

            XU = XU.to(device).float()
            outputsU = net(XU)
            yunppos = np.ones(BATCH_SIZE)
            yunpneg = -np.ones(BATCH_SIZE)
            yupos = torch.from_numpy(yunppos).to(device).float()
            yuneg = torch.from_numpy(yunpneg).to(device).float()

            
            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Loss
            loss = train_loss(outputs,ytr,smooth = False)

            # Unlabeled data
            lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
            loss+=Cu*lossunlab

            lam = torch.tensor(lam).to(device)
            l2_reg = torch.tensor(0.).to(device)

            for name, param in net.named_parameters():
                l2_reg += torch.norm(param)**2

            loss += lam * l2_reg

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1


        if epoch%50==0:

            idx_label_score = []
            net.eval()
            with torch.no_grad():
                for data in test_loader:
                    inputs, labels = data

                    labels = labels.cpu().data.numpy()
                    labels = (labels==normalclass).astype(int)

                    inputs = inputs.to(device)
                    outputs = net(inputs)
                    scores = outputs.data.cpu().numpy()
                    idx_label_score += list(zip(labels.tolist(),
                                                scores.tolist()))

            tstlabels, scores = zip(*idx_label_score)
            tstlabels = np.array(tstlabels)

            tstlabels[np.where(tstlabels==0)]=-1.0
            scores = np.array(scores)

            Tst_auc_score = evalmetrics(tstlabels,scores.flatten())

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))

            print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test AUC :{:.4f} '
                        .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_auc_score))


    # Final Solution
    logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

    print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   AUC (Test) :{:.4f} '
            .format(EPOCH, loss_epoch / n_batches, Tst_auc_score))

********************************
Normal class: 
9
lam = 0.05,lr = 0.0001, Cu = 0.05
Exp No: 0
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Files already downloaded and verified
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Epoch 0/350	 Time: 1.705	 Loss: 7.77868504 	 Test AUC :0.5627 
  Epoch 50/350	 Time: 1.691	 Loss: 6.80146448 	 Test AUC :0.7978 
  Epoch 100/350	 Time: 1.811	 Loss: 6.67330820 	 Test AUC :0.7994 
  Epoch 150/350	 Time: 1.686	 Loss: 6.54768683 	 Test AUC :0.8003 
  Epoch 200/350	 Time: 1.730	 Loss: 6.42442927 	 Test AUC :0.8010 
  Epoch 250/350	 Time: 1.713	 Loss: 6.30347317 	 Test AUC :0.8015 
  Epoch 300/350	 Time: 1.847	 Loss: 6.18480507 	 Test AUC :0.8016 
  Epoch 350/350	 Time: 1.662	 Loss: 6.06841050 	 Test AUC :0.8016 
 Final (TOT. EPOCHS 350)::  Loss: 6.06841050 	   AUC (Test) :0.8016 
